<a href="https://colab.research.google.com/github/RajaSuhashKesari/MyDataEngineeringPractices/blob/main/AvgsalesandRank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window as W

In [49]:
spark = SparkSession.builder.appName('test').getOrCreate()

In [50]:
data = [
    (1, 151),
    (1, 150),
    (1, 50),
    (1, 200),
    (1, 50),  # Duplicate of 50
    (2, 300),
    (2, 30111),
    (3, 150),
    (3, 150),
    (3, 152)
]

df = spark.createDataFrame(data, ["id", "sales"])
df.show()

+---+-----+
| id|sales|
+---+-----+
|  1|  151|
|  1|  150|
|  1|   50|
|  1|  200|
|  1|   50|
|  2|  300|
|  2|30111|
|  3|  150|
|  3|  150|
|  3|  152|
+---+-----+



# Average Sales by Id

In [51]:
avg_sales_per_id = df.groupBy("id").agg(F.avg("sales")).alias("avg_sales_per_id")
avg_sales_per_id.show()

+---+------------------+
| id|        avg(sales)|
+---+------------------+
|  1|             120.2|
|  3|150.66666666666666|
|  2|           15205.5|
+---+------------------+



In [59]:
wind_spec = W.orderBy("sales")

ranked_df = df.withColumn("Dense Rank",F.dense_rank().over(wind_spec))
ranked_df.show()

+---+-----+----------+
| id|sales|Dense Rank|
+---+-----+----------+
|  1|   50|         1|
|  1|   50|         1|
|  1|  150|         2|
|  3|  150|         2|
|  3|  150|         2|
|  1|  151|         3|
|  3|  152|         4|
|  1|  200|         5|
|  2|  300|         6|
|  2|30111|         7|
+---+-----+----------+



# Second Least Sales

In [65]:
second_least_sales = ranked_df.filter(F.col("Dense Rank") ==2)
second_least_sales.show()

+---+-----+----------+
| id|sales|Dense Rank|
+---+-----+----------+
|  1|  150|         2|
|  3|  150|         2|
|  3|  150|         2|
+---+-----+----------+

